In [5]:
import pandas as pd
print(pd)



<module 'pandas' from 'C:\\Users\\dfran\\Anaconda\\lib\\site-packages\\pandas\\__init__.py'>


In [6]:
data = pd.read_csv('patients.csv', index_col=0, na_values=(" ","  ","   ","    ","     "), low_memory=False)
data.columns = data.columns.str.strip()
# print first 10 rows
data.shape

(3900, 6)

In [7]:
data['ins'] = data['INSURANCE']
data['mar'] = data['MARITAL']
data['lang'] = data['LANGUAGE']
data['race'] = data['RACE']
data.head()

,PATIENT_ID,INSURANCE,RACE,LANGUAGE,MARITAL,LOS,ins,mar,lang,race
0,2,Private,ASIAN,NaN,NaN,4,Private,NaN,NaN,ASIAN
1,3,Medicare,WHITE,NaN,MARRIED,11,Medicare,MARRIED,NaN,WHITE
2,4,Private,WHITE,NaN,SINGLE,7,Private,SINGLE,NaN,WHITE
3,5,Private,ASIAN,NaN,NaN,2,Private,NaN,NaN,ASIAN
4,6,Medicare,WHITE,ENGL,MARRIED,16,Medicare,MARRIED,ENGL,WHITE


In [8]:
ins_type = data.INSURANCE.unique().tolist()
ins_type = [x for x in ins_type if str(x) != 'nan']
print(ins_type)

race_type = data.RACE.unique().tolist()
race_type = [x for x in race_type if str(x) != 'nan']
print(race_type)
lang_type = data.LANGUAGE.unique().tolist()
lang_type = [x for x in lang_type if str(x) != 'nan']
print(lang_type)
mar_type = data.MARITAL.unique().tolist()
mar_type = [x for x in mar_type if str(x) != 'nan']
print(mar_type)

['Private', 'Medicare', 'Medicaid', 'Government', 'Self Pay']
['ASIAN', 'WHITE', 'UNKNOWN/NOT SPECIFIED', 'BLACK/AFRICAN AMERICAN', 'PATIENT DECLINED TO ANSWER', 'OTHER', 'HISPANIC OR LATINO', 'HISPANIC/LATINO - GUATEMALAN', 'HISPANIC/LATINO - PUERTO RICAN', 'ASIAN - ASIAN INDIAN', 'ASIAN - VIETNAMESE', 'MULTI RACE ETHNICITY', 'HISPANIC/LATINO - DOMINICAN', 'AMERICAN INDIAN/ALASKA NATIVE', 'WHITE - RUSSIAN', 'HISPANIC/LATINO - SALVADORAN', 'ASIAN - CHINESE', 'UNABLE TO OBTAIN', 'BLACK/CAPE VERDEAN', 'WHITE - OTHER EUROPEAN', 'PORTUGUESE', 'BLACK/HAITIAN', 'SOUTH AMERICAN']
['ENGL', 'MAND', 'CANT', 'SPAN', 'GREE', 'CAMB', '*ARM', 'HAIT', 'RUSS', 'CAPE', 'PORT', '*MAN', 'VIET', 'ITAL', 'HIND', 'PTUN', '*CHI', 'ALBA', 'ARAB', 'POLI', 'FREN', 'AMER', 'LAOT', '*URD', '*PUN', '*BUR', '*BEN']
['MARRIED', 'SINGLE', 'WIDOWED', 'DIVORCED', 'SEPARATED', 'UNKNOWN (DEFAULT)']


In [9]:
new_data = data.copy()

#we are doing this the long way 
ins_dict= {}
race_dict = {}
lang_dict = {}
mar_dict = {}
for i in range(len(ins_type)):
    ins_dict[ins_type[i]] = i; 
for i in range(len(race_type)):
    race_dict[race_type[i]] = i; 
for i in range(len(lang_type)):
    lang_dict[lang_type[i]] = i; 
for i in range(len(mar_type)):
    mar_dict[mar_type[i]] = i; 

new_data.ins = [ins_dict[item] for item in new_data.ins]
new_data.race = [race_dict[item] for item in new_data.race]
#new_data.lang = [lang_dict[item] for item in new_data.lang]
#new_data.mar = [mar_dict[item] for item in new_data.mar]

for index, row in new_data.iterrows():
    if(str(row['mar']) == 'nan' and str(row['lang']) == 'nan'):
        continue 
    elif(str(row['lang']) == 'nan' and  not(str(row['mar']) == 'nan')):
        new_data.at[index, 'mar'] = mar_dict[row['mar']]
        continue 
    if(not(str(row['lang']) == 'nan') and str(row['mar']) == 'nan'):
        new_data.at[index, 'lang'] = lang_dict[row['lang']]
        continue 
    new_data.at[index, 'mar'] = mar_dict[row['mar']]
    new_data.at[index, 'lang'] = lang_dict[row['lang']]

sub_data = new_data.drop(columns = ['INSURANCE','RACE', 'LANGUAGE', 'MARITAL'])
sub_data.head()

cols = list(sub_data.columns.values)



In [10]:
from sklearn.preprocessing import Imputer

imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(sub_data)
imp_data = pd.DataFrame(imp.transform(sub_data.values))
#cols = ['EOD10_EX', 'EOD10_PN','EOD10_NE', 'RADIATN', 'GRADE','SEQ_NUM', 'RAC_RECA', 'AGE_DX', 'srv_time_mon', 'alive']
imp_data.columns = cols
imp_data.head()

C:\Users\dfran\Anaconda\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


,PATIENT_ID,LOS,ins,mar,lang,race
0,2.0,4.0,0.0,0.848897,2.318117,0.0
1,3.0,11.0,1.0,0.000000,2.318117,1.0
2,4.0,7.0,0.0,1.000000,2.318117,1.0
3,5.0,2.0,0.0,0.848897,2.318117,0.0
4,6.0,16.0,1.0,0.000000,0.000000,1.0


In [11]:
to_split =  imp_data
test_data = to_split.iloc[:3000, :]
train_data = to_split.iloc[3000:, :]

X_test  = test_data.iloc[:, 2:]
X_train =  train_data.iloc[:, 2:]

y1_train = train_data.iloc[:, 1:2]# YOUR CODE HERE
y1_test  = test_data.iloc[:, 1:2]# YOUR CODE HERE


In [12]:
import matplotlib.pyplot as plt

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y1_train)

# Make predictions using the testing set
y1_pred = regr.predict(X_test)
# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y1_test, y1_pred))

#Print the intercept and coefficients of the resulting model.
print (regr.intercept_)
print (regr.coef_)

Coefficients: 
 [[ 0.46761419  0.33009392 -0.07575373 -0.04153496]]
Variance score: -0.00
[10.07758311]
[[ 0.46761419  0.33009392 -0.07575373 -0.04153496]]


In [13]:
from math import sqrt
from sklearn.metrics import mean_squared_error, r2_score
print("Mean squared error: %.2f"
      % mean_squared_error(y1_test, y1_pred))
print("root mean squared error: %.2f"
      % sqrt(mean_squared_error(y1_test, y1_pred)))

Mean squared error: 211.86
root mean squared error: 14.56
